# Decision Trees

**CART** - Classification and Regression Trees

## Table of Contents

- [Introduction](#intro)
- [Entropy](#entro)
- [Example 1: Identify which candidates will interview well](#ex1)
- [Example 2: More general approach](#ex2)
- [CART - Classification and Regression Trees](#cart)

---
<a id='intro'></a>

## Introduction

A `decision tree` is a predictive modeling tool which uses a tree structure to represent a number of possible `decision paths` and an outcome for each path.

<img src="images/tree1.png" alt="" style="width: 600px;"/>


Decision trees have a lot to recommend them. They’re `very easy to understand and interpret`, and the process by which they reach a prediction is completely transparent. Unlike the other models, decision trees `can easily handle a mix of numeric` (e.g., number of legs) `and categorical` (e.g., delicious/not delicious) `attributes and can even classify data for which attributes are missing`.

At the same time, finding an “optimal” decision tree for a set of training data is computationally a very hard problem. (We will get around this by trying to build a goodenough tree rather than an optimal one, although for large data sets this can still be a lot of work.) More important, `it is very easy (and very bad) to build decision trees that are overfitted to the training data`, and that don’t generalize well to unseen data. We’ll look at ways to address this.

Most people divide decision trees into `classification trees` (which produce categorical outputs) and `regression trees` (which produce numeric outputs). 

---
<a id='entro'></a>

## Entropy

In order to build a `decision tree`, we will need to decide what questions to ask and in what order. At each stage of the tree there are some possibilities we’ve eliminated and some that we haven’t. Every possible question partitions the remaining possibilities according to their answers.

Ideally, `we’d like to choose questions whose answers give a lot of information about what our tree should predict`. If there’s a single yes/no question for which “yes” answers always correspond to True outputs and “no” answers to False outputs (or vice versa), this would be an awesome question to pick.

We capture this notion of “how much information” with `entropy` (think "uncertainty"). You have probably heard this used to mean disorder. We use it to represent the uncertainty associated with data.

Imagine that we have a set S of data, each member of which is labeled as belonging to one of a finite number of classes C1, ..., Cn. If all the data points belong to a single class, then there is no real uncertainty, which means we’d like there to be `low entropy`. If the data points are evenly spread across the classes, there is a lot of uncertainty and we’d like there to be `high entropy`.

<img src="images/tree2.png" alt="" style="width: 600px;"/>

Each term `−pi log2 pi` is non-negative and is close to zero precisely when pi is either close to zero or close to one.

<img src="images/tree3.png" alt="" style="width: 600px;"/>

This means the `entropy will be small` when every `pi` is close to 0 or 1 (i.e., when most of the data is in a single class), and it `will be larger` when many of the `pi`’s are not close to 0 (i.e., when the data is spread across multiple classes). 

In [11]:
from typing import List
import math

def entropy(class_probabilities: List[float]) -> float:
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p,2) 
               for p in class_probabilities
              if p > 0) # ignore zero probabilities

assert entropy([1.0]) == 0
assert entropy([0.5, 0.5]) == 1
assert 0.81 < entropy([0.25, 0.75]) < 0.82
assert entropy([0.25, 0.75]) == entropy([0.75, 0.25])

In [4]:
entropy([0.33, 0.33, 0.33])

1.5834674497121084

In [5]:
entropy([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

3.321928094887362

In [6]:
entropy([0.1, 0.1, 0.1, 0.1, 0.1, 0.5])

2.1609640474436813

In [8]:
entropy([0.1, 0.1, 0.2, 0.1, 0.1, 0.4])

2.3219280948873626

In [9]:
from typing import Any
from collections import Counter

def class_probabilities(labels: List[Any]) -> List[float]:
    """return the frewquency of class appearances in a list"""
    total_count = len(labels)
    return [count / total_count for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))

assert data_entropy(['a']) == 0
assert data_entropy(['True', 'False']) == 1
assert data_entropy([3, 4, 4, 4]) == entropy([0.25, 0.75])

Each stage of a decision tree involves asking a question whose answer partitions data into one or (hopefully) more subsets. We’d like some notion of the `entropy` that results from partitioning a set of data in a certain way. 

We want a partition to have:
- **low entropy** if it splits the data into subsets that themselves have low entropy (i.e., are highly certain), 
- and **high entropy** if it contains subsets that (are large and) have high entropy (i.e., are highly uncertain).

In [24]:
def partition_entropy(subsets: List[List[Any]]) -> float:
    """returns the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)
    return sum(data_entropy(subset) * len(subset) / total_count
              for subset in subsets)

One problem with this approach is that partitioning by an attribute with many different values will result in a `very low entropy due to overfitting`. For example, imagine you work for a bank and are trying to build a decision tree to predict which of your customers are likely to default on their mortgages, using some historical data as your training set. Imagine further that the data set contains each customer’s Social Security number. Partitioning on SSN will produce one-person subsets, each of which necessarily has zero entropy. But a model that relies on SSN is certain not to generalize beyond the training set. For this reason, you should probably `try to avoid (or bucket, if appropriate) attributes with large numbers of possible values when creating decision trees`.

---
<a id='ex1'></a>

## Example 1: Identify which candidates will interview well

In [14]:
from typing import NamedTuple, Optional

class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None # allow unlabeled data

In [15]:
                  #  level     lang     tweets  phd  did_well
inputs = [Candidate('Senior', 'Java',   False, False, False),
          Candidate('Senior', 'Java',   False, True,  False),
          Candidate('Mid',    'Python', False, False, True),
          Candidate('Junior', 'Python', False, False, True),
          Candidate('Junior', 'R',      True,  False, True),
          Candidate('Junior', 'R',      True,  True,  False),
          Candidate('Mid',    'R',      True,  True,  True),
          Candidate('Senior', 'Python', False, False, False),
          Candidate('Senior', 'R',      True,  False, True),
          Candidate('Junior', 'Python', True,  False, True),
          Candidate('Senior', 'Python', True,  True,  True),
          Candidate('Mid',    'Python', False, True,  True),
          Candidate('Mid',    'Java',   True,  False, True),
          Candidate('Junior', 'Python', False, True,  False)
         ]

Our tree will consist of `decision nodes` (which ask a question and direct us differently depending on the answer) and `leaf nodes` (which give us a prediction). We will build it using the relatively simple `ID3 algorithm`, which operates in the following manner. Let’s say we’re given some labeled data, and a list of attributes to consider branching on.

- If the data all have the same label, then create a `leaf node` that predicts that label and then stop.
- If the list of attributes is empty (i.e., there are no more possible questions to ask), then create a `leaf node` that predicts the most common label and then stop.
- Otherwise, try partitioning the data by each of the attributes
- Choose the partition with the lowest partition entropy
- Add a `decision node` based on the chosen attribute
- Recur on each partitioned subset using the remaining attributes

This is what’s known as a `“greedy” algorithm` because, at each step, it chooses the most immediately best option.

In [28]:
# Find the partition with the least entropy
# first create partitioning function
from typing import Dict, TypeVar
from collections import defaultdict

T = TypeVar('T') # generic type for inputs

def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    """Partition the inputs into lists based on the specified attribute."""
    partitions: Dict[Any, List[T]] = defaultdict(list)
    for input in inputs:
        key = getattr(input, attribute)  # value of the specified attribute
        partitions[key].append(input)    # add input to the correct partition
    return partitions

# then another for computing entropy
def partition_entropy_by(inputs: List[Any],
                         attribute: str,
                         label_attribute: str) -> float:
    """Compute the entropy corresponding to the given partition"""
    # partitions consist of our inputs
    partitions = partition_by(inputs, attribute)

    # but partition_entropy needs just the class labels
    labels = [[getattr(input, label_attribute) for input in partition]
              for partition in partitions.values()]

    return partition_entropy(labels)

# Then we just find the minimum-entropy partition for the whole data set:
for key in ['level','lang','tweets','phd']: 
    print(key, partition_entropy_by(inputs, key, 'did_well'))
    
assert 0.69 < partition_entropy_by(inputs, 'level', 'did_well')  < 0.70
assert 0.86 < partition_entropy_by(inputs, 'lang', 'did_well')   < 0.87
assert 0.78 < partition_entropy_by(inputs, 'tweets', 'did_well') < 0.79
assert 0.89 < partition_entropy_by(inputs, 'phd', 'did_well')    < 0.90

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


`The lowest entropy` comes from splitting on `level`, so we need to make a subtree for each possible `level` value. The next step is to follow the `Senior` branch

In [30]:
senior_inputs = [input for input in inputs if input.level == 'Senior']
#print(senior_inputs)

for key in ['level','lang','tweets','phd']: 
    print(key, partition_entropy_by(senior_inputs, key, 'did_well'))

assert 0.4 == partition_entropy_by(senior_inputs, 'lang', 'did_well')
assert 0.0 == partition_entropy_by(senior_inputs, 'tweets', 'did_well')
assert 0.95 < partition_entropy_by(senior_inputs, 'phd', 'did_well') < 0.96

level 0.9709505944546686
lang 0.4
tweets 0.0
phd 0.9509775004326938


In [34]:
# tweets have the lowest entropy
tweets_inputs = [input for input in senior_inputs if input.level == 'True']
print(tweets_inputs)

for key in ['level','lang','tweets','phd']: 
    print(key, partition_entropy_by(tweets_inputs, key, 'did_well'))

[]
level 0
lang 0
tweets 0
phd 0


In [36]:
senior_inputs = [input for input in inputs if input.level == 'Mid']
#print(senior_inputs)

for key in ['level','lang','tweets','phd']: 
    print(key, partition_entropy_by(senior_inputs, key, 'did_well'))

level 0.0
lang 0.0
tweets 0.0
phd 0.0


In [37]:
senior_inputs = [input for input in inputs if input.level == 'Junior']
#print(senior_inputs)

for key in ['level','lang','tweets','phd']: 
    print(key, partition_entropy_by(senior_inputs, key, 'did_well'))

level 0.9709505944546686
lang 0.9509775004326938
tweets 0.9509775004326938
phd 0.0


In [38]:
phd_inputs = [input for input in inputs if input.level == 'phd']
print(phd_inputs)

for key in ['level','lang','tweets','phd']: 
    print(key, partition_entropy_by(phd_inputs, key, 'did_well'))

[]
level 0
lang 0
tweets 0
phd 0


<img src="images/tree4.png" alt="" style="width: 600px;"/>


---
<a id='ex2'></a>

## Example 2: More general approach

In [39]:
from typing import NamedTuple, Union, Any

class Leaf(NamedTuple):
    value: Any

class Split(NamedTuple):
    attribute: str
    subtrees: dict
    default_value: Any = None

DecisionTree = Union[Leaf, Split]

hiring_tree = Split('level', {   # First, consider "level".
    'Junior': Split('phd', {     # if level is "Junior", next look at "phd"
        False: Leaf(True),       #   if "phd" is False, predict True
        True: Leaf(False)        #   if "phd" is True, predict False
    }),
    'Mid': Leaf(True),           # if level is "Mid", just predict True
    'Senior': Split('tweets', {  # if level is "Senior", look at "tweets"
        False: Leaf(False),      #   if "tweets" is False, predict False
        True: Leaf(True)         #   if "tweets" is True, predict True
    })
})

def classify(tree: DecisionTree, input: Any) -> Any:
    """classify the input using the given decision tree"""

    # If this is a leaf node, return its value
    if isinstance(tree, Leaf):
        return tree.value

    # Otherwise this tree consists of an attribute to split on
    # and a dictionary whose keys are values of that attribute
    # and whose values of are subtrees to consider next
    subtree_key = getattr(input, tree.attribute)

    if subtree_key not in tree.subtrees:   # If no subtree for key,
        return tree.default_value          # return the default value.

    subtree = tree.subtrees[subtree_key]   # Choose the appropriate subtree
    return classify(subtree, input)        # and use it to classify the input.

def build_tree_id3(inputs: List[Any],
                   split_attributes: List[str],
                   target_attribute: str) -> DecisionTree:
    # Count target labels
    label_counts = Counter(getattr(input, target_attribute)
                           for input in inputs)
    most_common_label = label_counts.most_common(1)[0][0]

    # If there's a unique label, predict it
    if len(label_counts) == 1:
        return Leaf(most_common_label)

    # If no split attributes left, return the majority label
    if not split_attributes:
        return Leaf(most_common_label)

    # Otherwise split by the best attribute

    def split_entropy(attribute: str) -> float:
        """Helper function for finding the best attribute"""
        return partition_entropy_by(inputs, attribute, target_attribute)

    best_attribute = min(split_attributes, key=split_entropy)

    partitions = partition_by(inputs, best_attribute)
    new_attributes = [a for a in split_attributes if a != best_attribute]

    # recursively build the subtrees
    subtrees = {attribute_value : build_tree_id3(subset,
                                                 new_attributes,
                                                 target_attribute)
                for attribute_value, subset in partitions.items()}

    return Split(best_attribute, subtrees, default_value=most_common_label)

tree = build_tree_id3(inputs,
                      ['level', 'lang', 'tweets', 'phd'],
                      'did_well')

# Should predict True
assert classify(tree, Candidate("Junior", "Java", True, False))

# Should predict False
assert not classify(tree, Candidate("Junior", "Java", True, True))

# Should predict True
assert classify(tree, Candidate("Intern", "Java", True, True))

---
<a id='cart'></a>

## CART - Classification and Regression Trees

The CART acronym was introduced by Leo Breiman to refer to Decision Tree algorithms that can be used for classification or regression predictive modeling problems. In this example we use a greedy approach called `recursive binary splitting`. This is a numerical procedure where all the values are lined up and different split points are tried and tested using a cost function. The split with the lowest cost is selected. All input variables and all split points are evaluated and chosen in a greedy manner based on the cost function.

- **Regression** - the cost function that is minimized to choose split points is the `Sum Squared Error (SSE)` accross all training samples that fall within the rectangle.
- **Classification** - the `Gini Index` cost function is used which provides an indication of how pure the nodes are, where the purity refers to how mixed the training data assigned to each node is.

Splitting continues until nodes contain a minimum number of training examples or a maximum tree depth is reached.

### Binary Classification

In [2]:
# Banknote Dataset: http://goo.gl/rLPrHO
# More information: http://archive.ics.uci.edu/ml/datasets/banknote+authentication
path = '../data/jbmla/'

### Gini Index

The `Gini Index` is the name of the cost function used to evaluate splits in the dataset. A Gini score gives an idea of how good a split is by how mixed the classes are in the two groups created by the split. A perfect separation results in a Gini score of 0, whereas the worst case split that results in 50/50 classes in each group results in a Gini score of 0.5 (for a 2 class problem).

<img src="images/jb-gini-weighted.png" alt="" style="width: 400px;"/>

Where `n` is number of classes.

In [88]:
from typing import List

def gini_index(groups:List, classes: List):
    '''
    Calculate the Gini index for a split dataset.
    :param A list of groups of observations.
    :param A list of known class values.
    '''
    # Count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    #print(f"n_instances: {n_instances}")
    
    # Sum weighted Gini index for each group
    gini = 0.0
    
    for group in groups:
        #print(f"group: {group}")
        
        size = float(len(group))
        #print(f"size: {size}")
        
        # Avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        
        # Score the group based on the score for each class
        for class_val in classes:
            #print(f"class_val: {class_val}")
            
            # Calculate the proportion of classes in each group
            # row[-1] - get the last element of a list, it gets a class assigned to an attribute
            proportion = [row[-1] for row in group].count(class_val) / size
            #print(f"proportion: {proportion}")
            
            score += proportion*proportion
            #print(f"score: {score}")
        
        gini += (1.0 - score) * (size / n_instances)
        #print(f"gini: {gini}")
            
    return gini

In [89]:
input_attr = 1
attr_value = 0 # to which class the attribute belongs
row1 = [input_attr, attr_value]
row2 = [1, 0]
group1 = [row1, 
          row2] # 2 first rows
group2 = [[0, 0], 
          [0, 0]] # 2 last rows
groups = [group1, group2]
clases = [0, 1]
gini_index(groups, clases)

0.0

In [92]:
# The worst Gini score is 0.5 (for binary classification)
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))

# The best Gini score is 0
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

0.5
0.0


### Create Split

Now that we know how to evaluate the results of a split, ;et's look at creating splits.